<a href="https://colab.research.google.com/github/Kridr/HW_OSDA_HSE/blob/main/HW4_OSDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Close-by-One algorithm from "Algorithms for the Construction of Concept Lattices and Their Diagram Graphs"

Literally magic library

In [1]:
import pandas as pd

Matrix of objects and attributes for Task 1

In [2]:
matrix = pd.DataFrame(
    columns=['a', 'b', 'c', 'd'],
    index=[1, 2, 3, 4, 5],
    data=[
        [0, 1, 1, 1],
        [1, 1, 1, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 1, 0, 1]
    ]
)

matrix

,a,b,c,d
1,0,1,1,1
2,1,1,1,0
3,0,1,1,0
4,1,0,0,1
5,1,1,0,1


Definition of primes function (or derivation operators)

In [3]:
def d(A, is_feat, df):
    """
    A: list of feature names or object names
    is_feat: whether list A contains feature names of object names
    df: object/feature matrix
    """
    if len(A) > 0:
        if not is_feat:
            answer = []
            for col in df.columns:
                subm = df.loc[A, col]
                if subm.sum() == subm.count():
                    answer.append(col)
        else:
            answer = []
            for ind in df.index:
                subm = df.loc[ind, A]
                if subm.sum() == subm.count():
                    answer.append(ind)
        return answer
    else:
        if is_feat:
            return list(df.index)
        else:
            return list(df.columns)

def dd(A, is_feat, df):
    return d(d(A, is_feat, df), not is_feat, df)

CbO algorithm from "Algorithms for the Construction of Concept Lattices and Their Diagram Graphs"

In [4]:
def Process(A, g, C, D, df):
    """
    additional parameter df: object/feature matrix
    """
    if not [h for h in C if h not in A and h < g]:
        L.append((C, D))
        for f in [h for h in df.index if h not in C and g < h]:
            Z = C + [f]
            dsf = d([f], False, df)
            Y = [x for x in D if x in dsf]
            X = d(Y, True, df)
            Process(Z, f, X, Y, df)

Concepts for Task 1

In [5]:
L = []
for g in matrix.index:
    Process([g], g, dd([g], False, matrix), d([g], False, matrix), matrix)
L

[([1], ['b', 'c', 'd']),
 ([1, 2, 3], ['b', 'c']),
 ([1, 2, 3, 4, 5], []),
 ([1, 2, 3, 5], ['b']),
 ([1, 4, 5], ['d']),
 ([1, 5], ['b', 'd']),
 ([2], ['a', 'b', 'c']),
 ([2, 4, 5], ['a']),
 ([2, 5], ['a', 'b']),
 ([4, 5], ['a', 'd']),
 ([5], ['a', 'b', 'd'])]

In [6]:
for l in L:
    print('(\{' + ', '.join([str(x) for x in l[0]]) + '\}, \{' + ', '.join(l[1]) + '\})\\newline')

(\{1\}, \{b, c, d\})\newline
(\{1, 2, 3\}, \{b, c\})\newline
(\{1, 2, 3, 4, 5\}, \{\})\newline
(\{1, 2, 3, 5\}, \{b\})\newline
(\{1, 4, 5\}, \{d\})\newline
(\{1, 5\}, \{b, d\})\newline
(\{2\}, \{a, b, c\})\newline
(\{2, 4, 5\}, \{a\})\newline
(\{2, 5\}, \{a, b\})\newline
(\{4, 5\}, \{a, d\})\newline
(\{5\}, \{a, b, d\})\newline


# Generator implication cover

Function for getting all subsets

In [7]:
import itertools

def get_combs(A, full=True):
    """
    A: list from which we need to find combinations
    full: flag; if full=True then A itself is included in the output combinations
    """
    combs = []
    finish = len(A) + 1 if full else len(A)
    for i in range(1, finish):
        combs += list(itertools.combinations(A, i))

    combs = [list(x) for x in combs]

    return combs

Util function for finding generator implication cover. Their names describe them well

In [8]:
def is_generator(B, D, df):
    return dd(B, True, df) == B and dd(D, True, df) == B

def is_minimal_generator(B, D, df):
    if is_generator(B, D, df):
        Es = get_combs(D, full=False)
        ddD = dd(D, True, df)
        for E in Es:
            if dd(E, True, df) == ddD:
                return False
        return True
    else:
        return False

def is_nontrivial_generator(B, D, df):
    return is_generator(B, D, df) and D != dd(D, True, df)

def nmingen(B, df):
    answer = []

    Ds = get_combs(B)
    for D in Ds:
        if is_minimal_generator(B, D, df) and is_nontrivial_generator(B, D, df):
            answer.append(D)

    return answer

Generator implication cover. I just used the definition from the slide

In [9]:
Bs = get_combs(matrix.columns)

GIC = []

for B in Bs:
    ddB = dd(B, True, matrix)
    if B in nmingen(ddB, matrix):
        Q = [x for x in ddB if x not in B]
        GIC.append((B, Q))

GIC

[(['c'], ['b']),
 (['a', 'c'], ['b']),
 (['c', 'd'], ['b']),
 (['a', 'c', 'd'], ['b'])]

# Proper premise base of implications

I just used the definition from the slides.

In [10]:
Bs = get_combs(matrix.columns)

PPB = []

for B in Bs:
    total = B.copy()
    if len(B) > 0:
        Bns = list(itertools.combinations(B, len(B) - 1))
        Bns = [list(x) for x in Bns]
        for Bn in Bns:
            total += dd(Bn, True, matrix)

        total = sorted(list(set(total)))
        ddB = dd(B, True, matrix)
        if ddB != total:
            PPB.append((B, ddB))

PPB

[(['c'], ['b', 'c'])]

# Minimum (Duquenne-Guigues) basis of implications

Defining inital matrix and converting it to binary  

In [11]:
m_sym = pd.DataFrame(
    columns=['a', 'b', 'c', 'd', 'e'],
    index=[1, 2, 3, 4, 5],
    data=[
        ['u', 'v', 't', 'r', 'q'],
        ['t', 't', 'q', 'r', 'q'],
        ['q', 't', 'q', 't', 't'],
        ['r', 'q', 'q', 'u', 'u'],
        ['q', 'v', 't', 'r', 't'],
    ]
)



pairs = itertools.combinations(m_sym.index, 2)
pair_to_bin = {}
for pair in pairs:
    name = str(pair[0]) + str(pair[1])
    pair_to_bin[name] = list((m_sym.loc[pair[0]] == m_sym.loc[pair[1]]).astype(int))

m_bin = pd.DataFrame.from_dict(pair_to_bin, orient='index')
m_bin.columns = m_sym.columns
m_bin

,a,b,c,d,e
12,0,0,0,1,1
13,0,0,0,0,0
14,0,0,0,0,0
15,0,1,1,1,0
23,0,1,1,0,0
24,0,0,1,0,0
25,0,0,0,1,0
34,0,0,1,0,0
35,1,0,0,0,1
45,0,0,0,0,0


In [12]:
print(m_bin.to_latex())
print()
print(m_sym.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  a &  b &  c &  d &  e \\
\midrule
12 &  0 &  0 &  0 &  1 &  1 \\
13 &  0 &  0 &  0 &  0 &  0 \\
14 &  0 &  0 &  0 &  0 &  0 \\
15 &  0 &  1 &  1 &  1 &  0 \\
23 &  0 &  1 &  1 &  0 &  0 \\
24 &  0 &  0 &  1 &  0 &  0 \\
25 &  0 &  0 &  0 &  1 &  0 \\
34 &  0 &  0 &  1 &  0 &  0 \\
35 &  1 &  0 &  0 &  0 &  1 \\
45 &  0 &  0 &  0 &  0 &  0 \\
\bottomrule
\end{tabular}


\begin{tabular}{llllll}
\toprule
{} &  a &  b &  c &  d &  e \\
\midrule
1 &  u &  v &  t &  r &  q \\
2 &  t &  t &  q &  r &  q \\
3 &  q &  t &  q &  t &  t \\
4 &  r &  q &  q &  u &  u \\
5 &  q &  v &  t &  r &  t \\
\bottomrule
\end{tabular}



Manual matrix simplification 

In [13]:
m_bin = m_bin.loc[['12', '15', '23', '24', '25', '35']]

Function that returns true if given subset of attributes is pseudointent

In [14]:
def pseudo_intent(P, df):
    ddP = dd(P, True, df)
    if P != ddP:
        if len(P) == 1:
            Qs = [[]]
        else:
            Qs = get_combs(P, False)
        for Q in Qs:
            if pseudo_intent(Q, df) and dd(Q, True, df) not in Qs:
                return False
        return True
    else:
        return False

We search for Minimum (Duquenne-Guigues) basis of implications using `pseudo_inten` function defined above

In [15]:
Fs = get_combs(m_bin.columns, True)

for F in Fs:
    if pseudo_intent(F, m_bin):
        print('$', ''.join(F), '\\rightarrow', ''.join(dd(F, True, m_bin)), '$\\newline')

$ a \rightarrow ae $\newline
$ b \rightarrow bc $\newline
$ cd \rightarrow bcd $\newline
$ ce \rightarrow abcde $\newline
$ ade \rightarrow abcde $\newline
